In [2]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Create app
app = dash.Dash(__name__)

# Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

# Read the wildfire data into pandas dataframe
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/'
                 'IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Historical_Wildfires.csv')

# Extract year and month from the date column
df['Month'] = pd.to_datetime(df['Date']).dt.month_name()  # month names
df['Year'] = pd.to_datetime(df['Date']).dt.year

# ===== Layout Section =====

# TASK 2.1: Add the Title to the Dashboard
app.layout = html.Div(children=[
    html.H1("Australian Wildfires Dashboard",
            style={'textAlign': 'center', 'color': 'darkred', 'font-size': 28}),

    # TASK 2.2: Add the radio items and a dropdown
    html.Div([
        # Helper text
        html.Div([
            html.H2("Select Region:", style={'margin-right': '2em'}),

            # Radio items for region
            dcc.RadioItems([
                {"label": "New South Wales", "value": "NSW"},
                {"label": "Victoria", "value": "VIC"},
                {"label": "Queensland", "value": "QLD"},
                {"label": "South Australia", "value": "SA"},
                {"label": "Western Australia", "value": "WA"},
                {"label": "Tasmania", "value": "TAS"},
                {"label": "Northern Territory", "value": "NT"}
            ],
                value="NSW", id='region-choice', inline=True)
        ]),

        # Dropdown for year
        html.Div([
            html.H2('Select Year:', style={'margin-right': '2em'}),
            dcc.Dropdown(
                id='year-choice',
                options=[{'label': y, 'value': y} for y in sorted(df['Year'].unique())],
                value=2019,
                placeholder="Select a year"
            )
        ], style={'width': '50%', 'display': 'inline-block'}),

        # TASK 2.3: Add two empty divisions for output
        html.Div([
            html.Div([], id='output-pie', className='six columns'),
            html.Div([], id='output-bar', className='six columns')
        ], style={'display': 'flex'})
    ])
])

# ===== Callbacks =====

# TASK 2.4: Add the Output and Input components
@app.callback(
    [Output(component_id='output-pie', component_property='children'),
     Output(component_id='output-bar', component_property='children')],
    [Input(component_id='region-choice', component_property='value'),
     Input(component_id='year-choice', component_property='value')]
)
# TASK 2.5: Define the callback function
def reg_year_display(input_region, input_year):
    # Filter by region and year
    region_data = df[df['Region'] == input_region]
    y_r_data = region_data[region_data['Year'] == input_year]

    # Monthly average estimated fire area
    est_data = y_r_data.groupby('Month')['Estimated_fire_area'].mean().reset_index()
    fig1 = px.pie(est_data, values='Estimated_fire_area', names='Month',
                  title=f"{input_region}: Monthly Average Estimated Fire Area in {input_year}")

    # Monthly average vegetation fire pixels
    veg_data = y_r_data.groupby('Month')['Count'].mean().reset_index()
    fig2 = px.bar(veg_data, x='Month', y='Count',
                  title=f"{input_region}: Average Count of Pixels for Presumed Vegetation Fires in {input_year}")

    return [dcc.Graph(figure=fig1), dcc.Graph(figure=fig2)]


# Run the app
if __name__ == '__main__':
    app.run(debug=True)
